En aquest codi analitzarem els resultats de cada partida i escriurem les classificacions corresponents.

In [20]:
# Importem les llibreries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr # per guardar les dades 3D
from collections import Counter

In [21]:
# Definim tab20 com la paleta per defecte dels plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab20.colors)

In [22]:
# Definim una funció que afegeix un nou matchday al ataframe
def join_matchdays(master_dataframe, dict_to_join):
     # Create dataframe with results of this matchday
    matchday_results = pd.DataFrame(dict_to_join.items()).transpose().reset_index(drop=True) # la llista vertical de resultats per jugador, la passem a fila
    matchday_results.columns = matchday_results.iloc[0] # definim que els noms de la columna són els noms dels jugadors (que surten a la 1a fila)
    matchday_ratio = matchday_results.drop(matchday_results.index[0]) #esborrem la primera fila, que conté els noms del jugadors

    # Agrupem els resultats d'aquesta jornada amb els de les anteriors (columna = nom jugador; fila = matchday)
    master_dataframe = pd.concat([master_dataframe, matchday_ratio], ignore_index=True)

    return master_dataframe

In [23]:
# Carreguem les dades
data_df = pd.read_csv('results.csv')

# Emplenem els espais en blanc amb 0
data_df = data_df.fillna(0.)

In [24]:
# Obtenim una llista amb tots els noms dels participants
players_names = np.unique(data_df[['Jugador 1', 'Jugador 2', 'Jugador 3', 'Jugador 4']].values.flatten())

# Llista de dies jugats
matchdays = pd.unique(data_df['D'])

In [25]:
# Comptem quants partits ha jugat cada participant
all_players = data_df['Jugador 1'].tolist() + data_df['Jugador 2'].tolist() + data_df['Jugador 3'].tolist() + data_df['Jugador 4'].tolist()

games_count = dict(Counter(all_players))


In [26]:
# En aquest dataframe hi guardem les estadístiques finals després de cada jornada
winplayed_matchdays = pd.DataFrame(columns=players_names)
played_matchdays = pd.DataFrame(columns=players_names)

for nmatchday in range(len(matchdays)):
    # Initialize an empty dictionary to store data
    win_counts = {}
    played_counts = {}
    winplayed_counts = {}
    
    for player in players_names: # set all initial wins to 0
        win_counts[player] = 0

    # Select matchdays
    matchday_df = data_df.loc[data_df['D'] <= nmatchday+1]
    
    # Home wins
    home_wins = matchday_df[matchday_df['Local'] > matchday_df['Visitant']]
    for player in set(home_wins['Jugador 1'].tolist() + home_wins['Jugador 2'].tolist()):
        win_counts[player] = win_counts.get(player, 0) + 1
    
    # Away wins
    away_wins = matchday_df[matchday_df['Visitant'] > matchday_df['Local']]
    for player in set(away_wins['Jugador 3'].tolist() + away_wins['Jugador 4'].tolist()):
        win_counts[player] = win_counts.get(player, 0) + 1

    # Games played
    for player in players_names:
        # Comptem quantes vegades el nom del jugador apareix al registre de partits
        games_played = (matchday_df[['Jugador 1', 'Jugador 2', 'Jugador 3', 'Jugador 4']] == player).sum().sum()

        # Desem a un diccionari el recompte de partits jugats
        played_counts[player] = games_played

        # Desem a un diccionari la ràtio entre partits guanyats i partits jugats
        if games_played == 0:
            winplayed_counts[player] = 0
        else:
            winplayed_counts[player] = win_counts.get(player, 0) / games_played
        
        #print(player)
        #win_counts[player] = win_counts.get(player, 0) / 

       
    # Agrupem els resultats d'aquesta jornada amb els de les anteriors (columna = nom jugador; fila = matchday)
    winplayed_matchdays = join_matchdays(winplayed_matchdays, winplayed_counts)
    played_matchdays = join_matchdays(played_matchdays, played_counts)

# Convert to DataFrame for display
#win_counts_df = pd.concat(pd:winDataFrame(list(win_counts.items()), columns=['Player', 'WinCount'])
# played_matchdays
#win_counts_df

In [27]:
set(data_df['Jugador 1'].tolist())

{'Alejandro',
 'Antía',
 'Clàudia',
 'Dani',
 'Guille',
 'Luis',
 'Maider',
 'Pau',
 'Pedro',
 'Víctor'}

### Gols anotats

In [28]:
played_matchdays

,Alejandro,Antía,Clàudia,Dani,Elena,Guille,Luis,Maider,Pablo,Pau,Pedro,Rebeca,Simone,Tiziano,Víctor
0,0,6,0,6,0,6,6,0,0,0,0,0,0,0,0
1,0,10,0,10,0,6,10,0,0,0,0,0,0,0,4
2,0,12,2,15,0,10,15,2,0,5,0,0,1,2,4
3,3,12,2,15,0,10,20,5,0,9,4,0,1,2,9
4,3,12,2,21,0,16,26,11,0,9,4,0,1,2,9
5,3,12,2,21,0,22,32,17,0,9,10,0,1,2,9
6,3,12,2,27,0,22,38,23,0,15,10,0,1,2,9
7,3,16,2,36,0,31,46,29,0,19,10,0,1,2,9
8,3,20,2,40,4,31,46,34,2,19,10,5,1,2,9
9,3,20,2,46,4,37,46,34,2,25,16,5,1,2,9


In [35]:
# En aquest dataframe hi guardem les estadístiques finals després de cada jornada
scored_matchdays = pd.DataFrame(columns=players_names)
scoredplayed_matchdays = pd.DataFrame(columns=players_names)

for nmatchday in range(len(matchdays)):
    # Initialize an empty dictionary to store data
    scored_counts = {}
    scoredplayed_counts = {}
    
    for player in players_names: # set all initial wins to 0
        scored_counts[player] = 0
        scoredplayed_counts[player] = 0

    # Select matchdays
    matchday_df = data_df.loc[data_df['D'] <= nmatchday+1]

    # Select the dataframe index for the last recorded game of this matchday
    # last_matchday_index = matchday_df['D'].index.max()
    
    # Scorded home defense
    for player in set(matchday_df['Jugador 1'].tolist()):
        scored_counts[player] = scored_counts.get(player, 0) + matchday_df['Gols 1'][matchday_df['Jugador 1'] == player].sum() # socred goals
    # Scorded home attack
    for player in set(matchday_df['Jugador 2'].tolist()):
        scored_counts[player] = scored_counts.get(player, 0) + matchday_df['Gols 2'][matchday_df['Jugador 2'] == player].sum()
    # Scorded away defense
    for player in set(matchday_df['Jugador 3'].tolist()):
        scored_counts[player] = scored_counts.get(player, 0) + matchday_df['Gols 3'][matchday_df['Jugador 3'] == player].sum()
    # Scorded away attack
    for player in set(matchday_df['Jugador 4'].tolist()):
        scored_counts[player] = scored_counts.get(player, 0) + matchday_df['Gols 4'][matchday_df['Jugador 4'] == player].sum()

    # Ratio scored / played for each player
    for player in set(matchday_df['Jugador 1'].tolist() + matchday_df['Jugador 2'].tolist() + matchday_df['Jugador 3'].tolist() + matchday_df['Jugador 4'].tolist()):   
        print(player, played_matchdays[player].iloc[-1])
        scoredplayed_counts[player] = scored_counts.get(player, 0) / played_matchdays[player].iloc[nmatchday] # ratio scored / played
    
    
    # Agrupem els resultats d'aquesta jornada amb els de les anteriors (columna = nom jugador; fila = matchday)
    scored_matchdays = join_matchdays(scored_matchdays, scored_counts)
    scoredplayed_matchdays = join_matchdays(scoredplayed_matchdays, scoredplayed_counts)


Luis 55
Guille 46
Dani 54
Antía 35
Luis 55
Guille 46
Antía 35
Dani 54
Víctor 14
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Maider 44
Clàudia 2
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Pablo 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Rebeca 9
Maider 44
Clàudia 2
Elena 5
Antía 35
Pau 39
Guille 46
Luis 55
Tiziano 2
Pablo 2
Víctor 14
Dani 54
Simone 1
Alejandro 3
Pedro 17
Rebeca 9
Maider 44
Clàudia 2
Elena 5
Antí

Desem les dades a un xarray. Aquest format permet emmagatzemar matrius 3D, cosa que pandas no ho permet. A la nostra matriu tindrem dimensions (Nom de jugador, Dia de partit, Paràmetre). Això ens permet accedir a l'element que deseitgem.

In [ ]:
# Creem una DataArray de xarray. Hi especifiquem els noms de cada dimensió
winplayed_matchdays_da = xr.DataArray(winplayed_matchdays.values, dims = ('matchday', 'player'),
                                      coords = {'matchday': winplayed_matchdays.index, 'player': winplayed_matchdays.columns})
played_matchdays_da = xr.DataArray(played_matchdays.values, dims = ('matchday', 'player'),
                                      coords = {'matchday': played_matchdays.index, 'player': played_matchdays.columns})
scored_matchdays_da = xr.DataArray(scored_matchdays.values, dims = ('matchday', 'player'),
                                      coords = {'matchday': scored_matchdays.index, 'player': scored_matchdays.columns})
scoredplayed_matchdays_da = xr.DataArray(scoredplayed_matchdays.values, dims = ('matchday', 'player'),
                                      coords = {'matchday': scoredplayed_matchdays.index, 'player': scoredplayed_matchdays.columns})



# Combinem tots els DataArrays a un únic Dataset de xarray (cal que les coords siguin les mateixes per a tots)
dataset = xr.Dataset({"GamesPlayed": played_matchdays_da,
                      "WinPlayed": winplayed_matchdays_da,
                      "Scored": scored_matchdays_da,
                      "ScoredPlayed": scoredplayed_matchdays_da})
# TODO: el procés de crear el DataArray a partir del DataFrame es pot automatitzar amb una funció que faci un concat al dataframe. 

# dataset['goals'] = goals_da # si volem afegir un nou element
dataset

## Variables per desar a xarray:
- Partits jugats
- Gols anotats
- Gols rebuts
- Gols anotats atacant
- Gols anotats defensor
- Gols anotats atacant local
- Gols anotats atacant visitant
- Gols anotats defensor local
- Gols anotats defensor visitant
- Gols rebuts atacant
- Gols rebuts defensor
- Gols rebuts atacant local
- Gols rebuts atacant visitant
- Gols rebuts defensor local
- Gols rebuts defensor visitant
- Un paràmetre d'atacant (que ponderi contra qui s'està jugant)
- Un paràmetre de defensa (que ponderi contra qui s'està defensant)

## Una nova xarray
- Una nova xarray on s'inclogui la freqüència d'ocurrència de cada parella. La 3a dimensió seria cada jornada, i les dues dimensions principals serien els noms dels jugadors. És la matriu amb 0 a la diagonal que teníem abans.
- Igual que l'anterior, però amb freqüència de victòries.

## Per pensar:
- Intentar trobar la manera de saber com es pot saber quina parella guanya més partits contra qui.

Desem el fitxer xarray en format netcdf4. Això ens permetrà obrir-lo amb un altre fitxer i fer-ne l'anàlisi que volguem.

In [ ]:
dataset.to_netcdf('stats.nc', mode='w')
# ds = xr.open_dataset('stats.nc', engine ='netcdf4') # si volem obrir el fitxer